<a href="https://colab.research.google.com/github/Shaheer-khan-github/MLOps-Model-Development/blob/main/01_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/tcapelle/edu/blob/main/mlops-001/lesson1/01_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{course-lesson1} -->

# EDA 
<!--- @wandbcode{course-lesson1} -->

In this notebook, we will download a sample of the [BDD100K](https://www.bdd100k.com/) semantic segmentation dataset and use W&B Artifacts and Tables to version and analyze our data. 

In [ ]:
DEBUG = False # set this flag to True to use a small subset of data for testing

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd drive/MyDrive/Weights-and-Biases-MLops/mlops-001/lesson1

[Errno 2] No such file or directory: 'drive/MyDrive/Weights-and-Biases-MLops/mlops-001/lesson1'
/content/drive/MyDrive/Weights-and-Biases-MLops/mlops-001/lesson1


In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from fastai.vision.all import *
import params

import wandb

We have defined some global configuration parameters in the `params.py` file. `ENTITY` should correspond to your W&B Team name if you work in a team, replace it with `None` if you work individually. 

In the section below, we will use `untar_data` function from `fastai` to download and unzip our datasets. 

In [ ]:
URL = 'https://storage.googleapis.com/wandb_course/bdd_simple_1k.zip'

In [ ]:
path = Path(untar_data(URL, force_download=True))

In [ ]:
path.ls()

(#3) [Path('/root/.fastai/data/bdd_simple_1k/LICENSE.txt'),Path('/root/.fastai/data/bdd_simple_1k/labels'),Path('/root/.fastai/data/bdd_simple_1k/images')]

Here we define several functions to help us process the data and upload it as a `Table` to W&B. 

In [ ]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_classes_per_image(mask_data, class_labels):
    unique = list(np.unique(mask_data))
    result_dict = {}
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    return result_dict

def _create_table(image_files, class_labels):
    "Create a table with the dataset"
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    table = wandb.Table(columns=["File_Name", "Images", "Split"] + labels)
    
    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        mask_data = np.array(Image.open(label_func(image_file)))
        class_in_image = get_classes_per_image(mask_data, class_labels)
        table.add_data(
            str(image_file.name),
            wandb.Image(
                    image,
                    masks={
                        "predictions": {
                            "mask_data": mask_data,
                            "class_labels": class_labels,
                        }
                    }
            ),
            "None", # we don't have a dataset split yet
            *[class_in_image[_lab] for _lab in labels]
        )
    
    return table

We will start a new W&B `run` and put everything into a raw Artifact.

In [ ]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="upload")
raw_data_at = wandb.Artifact(params.RAW_DATA_AT, type="raw_data")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
print(path)

/root/.fastai/data/bdd_simple_1k


In [ ]:
raw_data_at.add_file(path/'LICENSE.txt', name='LICENSE.txt')

<ManifestEntry digest: X+6ZFkDOlnKesJCNt20yRg==>

Let's add the images and label masks.

In [ ]:
raw_data_at.add_dir(path/'images', name='images')
raw_data_at.add_dir(path/'labels', name='labels')

wandb: Adding directory to artifact (/root/.fastai/data/bdd_simple_1k/images)... Done. 2.3s
wandb: Adding directory to artifact (/root/.fastai/data/bdd_simple_1k/labels)... Done. 1.4s


Let's get the file names of images in our dataset and use the function we defined above to create a W&B Table. 

In [ ]:
image_files = get_image_files(path/"images", recurse=False)

# sample a subset if DEBUG
if DEBUG: image_files = image_files[:10]

In [ ]:
table = _create_table(image_files, params.BDD_CLASSES)

Finally, we will add the Table to our Artifact, log it to W&B and finish our `run`. 

In [ ]:
raw_data_at.add(table, "eda_table")

<ManifestEntry digest: hxIdZcGnIPqGZpQifdLQcA==>

In [ ]:
run.log_artifact(raw_data_at)
run.finish()